In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2

        # Rescale
        img = np.dstack([band_1,band_2,band_3])
        img = std_img(img)

        images.append(img)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [4]:
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()
    model.add(Conv2D(16, (3, 3), padding='same',input_shape=(75,75,3)))
    model.add(Activation('relu'))
    model.add(Conv2D(16, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    
    return model
print('model model')


model model


In [5]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator

def lr_f(epoch):
    if epoch<10:
        return 0.001
    elif epoch<30:
        return 0.0005
    else:
        return 0.0001

def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
           width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=50, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

Epoch 1/50
Epoch 00001: val_loss improved from inf to 0.66110, saving model to best_m.h5
 - 2s - loss: 0.6726 - acc: 0.5658 - val_loss: 0.6611 - val_acc: 0.6933
Epoch 2/50
Epoch 00002: val_loss improved from 0.66110 to 0.64933, saving model to best_m.h5
 - 1s - loss: 0.6029 - acc: 0.6458 - val_loss: 0.6493 - val_acc: 0.5636
Epoch 3/50
Epoch 00003: val_loss improved from 0.64933 to 0.50257, saving model to best_m.h5
 - 1s - loss: 0.5415 - acc: 0.7075 - val_loss: 0.5026 - val_acc: 0.7332
Epoch 4/50
Epoch 00004: val_loss improved from 0.50257 to 0.49832, saving model to best_m.h5
 - 1s - loss: 0.5172 - acc: 0.7372 - val_loss: 0.4983 - val_acc: 0.8304
Epoch 5/50
Epoch 00005: val_loss improved from 0.49832 to 0.42481, saving model to best_m.h5
 - 1s - loss: 0.4606 - acc: 0.7841 - val_loss: 0.4248 - val_acc: 0.8304
Epoch 6/50
Epoch 00006: val_loss improved from 0.42481 to 0.40914, saving model to best_m.h5
 - 1s - loss: 0.4689 - acc: 0.7858 - val_loss: 0.4091 - val_acc: 0.8080
Epoch 7/50
Epo

Epoch 10/50
Epoch 00010: val_loss improved from 0.39253 to 0.39250, saving model to best_m.h5
 - 1s - loss: 0.3362 - acc: 0.8508 - val_loss: 0.3925 - val_acc: 0.7955
Epoch 11/50
Epoch 00011: val_loss improved from 0.39250 to 0.32385, saving model to best_m.h5
 - 2s - loss: 0.3373 - acc: 0.8504 - val_loss: 0.3238 - val_acc: 0.8429
Epoch 12/50
Epoch 00012: val_loss did not improve
 - 1s - loss: 0.3261 - acc: 0.8583 - val_loss: 0.3464 - val_acc: 0.8354
Epoch 13/50
Epoch 00013: val_loss improved from 0.32385 to 0.31612, saving model to best_m.h5
 - 1s - loss: 0.2933 - acc: 0.8758 - val_loss: 0.3161 - val_acc: 0.8653
Epoch 14/50
Epoch 00014: val_loss improved from 0.31612 to 0.30569, saving model to best_m.h5
 - 1s - loss: 0.3011 - acc: 0.8600 - val_loss: 0.3057 - val_acc: 0.8603
Epoch 15/50
Epoch 00015: val_loss did not improve
 - 1s - loss: 0.2923 - acc: 0.8792 - val_loss: 0.3077 - val_acc: 0.8653
Epoch 16/50
Epoch 00016: val_loss did not improve
 - 1s - loss: 0.2952 - acc: 0.8725 - val_l

Epoch 20/50
Epoch 00020: val_loss did not improve
 - 1s - loss: 0.3205 - acc: 0.8522 - val_loss: 0.2612 - val_acc: 0.8828
Epoch 21/50
Epoch 00021: val_loss did not improve
 - 1s - loss: 0.2838 - acc: 0.8733 - val_loss: 0.2556 - val_acc: 0.8703
Epoch 22/50
Epoch 00022: val_loss did not improve
 - 1s - loss: 0.3089 - acc: 0.8581 - val_loss: 0.2222 - val_acc: 0.9002
Epoch 23/50
Epoch 00023: val_loss improved from 0.21291 to 0.20001, saving model to best_m.h5
 - 2s - loss: 0.2792 - acc: 0.8842 - val_loss: 0.2000 - val_acc: 0.9077
Epoch 24/50
Epoch 00024: val_loss improved from 0.20001 to 0.19559, saving model to best_m.h5
 - 2s - loss: 0.3029 - acc: 0.8689 - val_loss: 0.1956 - val_acc: 0.9052
Epoch 25/50
Epoch 00025: val_loss did not improve
 - 1s - loss: 0.3085 - acc: 0.8606 - val_loss: 0.2211 - val_acc: 0.9102
Epoch 26/50
Epoch 00026: val_loss did not improve
 - 1s - loss: 0.2769 - acc: 0.8773 - val_loss: 0.2396 - val_acc: 0.8903
Epoch 27/50
Epoch 00027: val_loss did not improve
 - 1s - 

Epoch 29/50
Epoch 00029: val_loss did not improve
 - 2s - loss: 0.2593 - acc: 0.8908 - val_loss: 0.2916 - val_acc: 0.8653
Epoch 30/50
Epoch 00030: val_loss did not improve
 - 1s - loss: 0.2385 - acc: 0.9017 - val_loss: 0.3193 - val_acc: 0.8479
Epoch 31/50
Epoch 00031: val_loss did not improve
 - 1s - loss: 0.2336 - acc: 0.8908 - val_loss: 0.2843 - val_acc: 0.8728
Epoch 32/50
Epoch 00032: val_loss did not improve
 - 1s - loss: 0.2148 - acc: 0.9100 - val_loss: 0.2885 - val_acc: 0.8703
Epoch 33/50
Epoch 00033: val_loss did not improve
 - 2s - loss: 0.2259 - acc: 0.8995 - val_loss: 0.2919 - val_acc: 0.8703
Epoch 34/50
Epoch 00034: val_loss did not improve
 - 1s - loss: 0.2286 - acc: 0.8914 - val_loss: 0.2924 - val_acc: 0.8678
Epoch 35/50
Epoch 00035: val_loss improved from 0.28356 to 0.28256, saving model to best_m.h5
 - 1s - loss: 0.2123 - acc: 0.9058 - val_loss: 0.2826 - val_acc: 0.8653
Epoch 36/50
Epoch 00036: val_loss did not improve
 - 1s - loss: 0.2092 - acc: 0.9117 - val_loss: 0.286

In [6]:
import pickle
with open('../features/cnn_1_aug1_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_1_aug1_sub.csv', index=False)

# pre 2508
# new 2239

0.223983930115
         id  is_iceberg
0  5941774d    0.098441
1  4023181e    0.986018
2  b20200e4    0.867378
3  e7f018bb    0.993455
4  4371c8c3    0.964587
